In [16]:
from astropy.time import Time
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from lasair import LasairError, lasair_consumer, lasair_client as lasair
import random, json
import settings 

In [17]:
def handle_filter_row(L, filter_row, showAttr):
    jdnow         = Time.now().jd
    objectId      = filter_row['objectId']
    objectData    = L.objects([objectId])[0]
    candidates    = objectData['candidates']
    colourFromFid = {1:'green', 2:'red', 3:'blue'}
    
    last_candidate = None
    for c in candidates:
        if not 'candid' in c: 
            continue
        last_candidate = c
        break
    s = '<h2><a href=%s/objects/%s>%s</a></h2>'
    s = s % (lasairUrl, objectId, objectId)
    display(HTML(s))
    
    t = Time(last_candidate['jd'], format='jd')
    iso = t.to_value('iso', subfmt='date_hm')
    s = 'Cutouts from %s' % iso
    if 'TNS' in objectData:
        tns = objectData['TNS']
        if 'tns_name' in tns:
            tnsfmt = '&nbsp;&nbsp;|&nbsp;&nbsp;in TNS: <b><a href=https://www.wis-tns.org/object/%s>%s</a></b>'
            s += tnsfmt % (tns['tns_name'], tns['tns_name'])
        if 'type' in tns:
            s += ', type=%s' % tns['type']
    display(HTML(s))
    
    # Set up the horizontal subplots
    fig, axs = plt.subplots(ncols=5, figsize=(15, 3), 
        gridspec_kw={'width_ratios': [2, 4, 2, 2, 2]})
    
    # Table of attributes
    showAttributes(axs[0], filter_row, showAttr)
    
    # Image cutouts if available
    if "image_urls" in last_candidate:
        show_fits(
            axs[2], last_candidate["image_urls"]["Template"],
            axs[3], last_candidate["image_urls"]["Science"],
        )

    # PS1 image
    showPS1(axs[4], objectData['objectData'])

    # Lightcurve
    ax = axs[1]
    for c in candidates:
        if not 'candid' in c: 
            continue
        if not last_candidate:
            last_candidate = c
        if 'isdiffpos' in c:
            if c['isdiffpos']=='t':
                ax.errorbar(c['jd']-jdnow,c['magpsf'],c['sigmapsf'],
                    fmt='o',color=colourFromFid[c['fid']])
    ax.invert_yaxis()
    ax.set_xlabel('Days ago')
    ax.set_xlim(right=0)
    ax.set_title(objectId)
    plt.show()
    
    # Checkboxes for veto/fave
    showCheckboxes(objectId)

In [18]:
def run(token, kafka_server, group_id, my_topic, alertsPerPage, showAttr):
    L = lasair(token, endpoint=lasairUrl+'/api')
    readFiles(vetoFile, faveFile, verbose=False)
    consumer = lasair_consumer(kafka_server, group_id, my_topic)

    n = 0
    while n < alertsPerPage:
        msg = consumer.poll(timeout=20)
        if msg is None:
            break
        if msg.error():
            print(str(msg.error()))
            break
        filter_row = json.loads(msg.value())
        display(HTML('<hr style="height:5px;">'))
        objectId = filter_row['objectId']
        if objectId in vetoList:
            s = '<a href=%s/objects/%s>%s</a> vetoed: %s'
            s = s % (lasairUrl, objectId, objectId, vetoList[objectId])
            display(HTML(s))
            continue
        handle_filter_row(L, filter_row, showAttr)
        n += 1
    display(HTML('<hr style="height:5px;"> Displayed %d filter rows. Run all cells to get more' % n))

    button = Button(description = "Save veto/fave choices")
    display(button)
    button.on_click(on_button_clicked)